In [29]:
import pandas as pd
import pandas_ta as ta
from tqdm import tqdm
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
from backtesting import Strategy
from backtesting import Backtest

### 1 - Import test data

In [30]:
df = pd.read_csv("data.csv")
df["Gmt time"]=df["Gmt time"].str.replace(".000","")
df['Gmt time']=pd.to_datetime(df['Gmt time'], format='%d.%m.%Y %H:%M:%S')
df=df[df.High!=df.Low] # Closing days
df.set_index("Gmt time", inplace=True)

In [31]:
df["EMA_slow"]=ta.ema(df.Close, length=50)
df["EMA_fast"]=ta.ema(df.Close, length=30)
df['RSI']=ta.rsi(df.Close, length=10)
my_bbands = ta.bbands(df.Close, length=15, std=1.5)
df['ATR']=ta.atr(df.High, df.Low, df.Close, length=7)
df=df.join(my_bbands)

In [32]:
df.shape

(224989, 14)

In [33]:
def ema_signal(df, current_candle, backcandles):
    df_slice = df.reset_index().copy()
    # Get the range of candles to consider
    start = max(0, current_candle - backcandles)
    end = current_candle
    relevant_rows = df_slice.iloc[start:end]

    # Check if all EMA_fast values are below EMA_slow values
    if all(relevant_rows["EMA_fast"] < relevant_rows["EMA_slow"]):
        return 1
    elif all(relevant_rows["EMA_fast"] > relevant_rows["EMA_slow"]):
        return 2
    else:
        return 0

df=df[-10000:-1]
tqdm.pandas()
df.reset_index(inplace=True)
df['EMASignal'] = df.progress_apply(lambda row: ema_signal(df, row.name, 7) , axis=1) #if row.name >= 20 else 0

100%|██████████| 9999/9999 [00:13<00:00, 720.92it/s]


In [34]:
def total_signal(df, current_candle, backcandles):
    if (ema_signal(df, current_candle, backcandles)==2
        and df.Close[current_candle]<=df['BBL_15_1.5'][current_candle]
        #and df.RSI[current_candle]<60
        ):
            return 2
    if (ema_signal(df, current_candle, backcandles)==1
        and df.Close[current_candle]>=df['BBU_15_1.5'][current_candle]
        #and df.RSI[current_candle]>40
        ):
    
            return 1
    return 0
        
df['TotalSignal'] = df.progress_apply(lambda row: total_signal(df, row.name, 7), axis=1)

100%|██████████| 9999/9999 [00:31<00:00, 312.65it/s]


In [35]:
df[df.TotalSignal != 0].head()

,Gmt time,Open,High,Low,Close,Volume,EMA_slow,EMA_fast,RSI,ATR,BBL_15_1.5,BBM_15_1.5,BBU_15_1.5,BBB_15_1.5,BBP_15_1.5,EMASignal,TotalSignal
3,2022-08-15 03:25:00,1.02478,1.02492,1.02474,1.02487,723.19,1.025308,1.025030,51.979645,0.000268,1.024246,1.024556,1.024866,0.060462,1.006890,1,1
7,2022-08-15 03:45:00,1.02491,1.02523,1.02489,1.02508,569.41,1.025250,1.025005,59.462538,0.000257,1.024224,1.024621,1.025019,0.077612,1.076775,1,1
8,2022-08-15 03:50:00,1.02508,1.02525,1.02504,1.02525,370.81,1.025250,1.025021,64.530972,0.000251,1.024225,1.024677,1.025129,0.088246,1.134054,1,1
9,2022-08-15 03:55:00,1.02523,1.02533,1.02518,1.02523,591.27,1.025249,1.025034,63.493245,0.000236,1.024276,1.024741,1.025206,0.090773,1.026057,1,1
11,2022-08-15 04:05:00,1.02519,1.02534,1.02507,1.02534,740.19,1.025250,1.025063,65.954393,0.000244,1.024458,1.024878,1.025298,0.081879,1.050553,1,1


In [36]:
def pointpos(x):
    if x['TotalSignal']==2:
        return x['Low']-1e-3
    elif x['TotalSignal']==1:
        return x['High']+1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [37]:
st=100
dfpl = df[st:st+350]
#dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),

                go.Scatter(x=dfpl.index, y=dfpl['BBL_15_1.5'], 
                           line=dict(color='green', width=1), 
                           name="BBL"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_15_1.5'], 
                           line=dict(color='green', width=1), 
                           name="BBU"),
                go.Scatter(x=dfpl.index, y=dfpl['EMA_fast'], 
                           line=dict(color='black', width=1), 
                           name="EMA_fast"),
                go.Scatter(x=dfpl.index, y=dfpl['EMA_slow'], 
                           line=dict(color='blue', width=1), 
                           name="EMA_slow")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="entry")

fig.show()

In [38]:
def SIGNAL():
    return df.TotalSignal

In [42]:
class MyStrategy(Strategy):
    mysize = 3000
    slcoef = 1.1
    TPSLRatio = 1.5
    rsi_length = 16
    
    def init(self):
        super().init()
        self._signal = self.I(SIGNAL)
        #df['RSI']=ta.rsi(df.Close, length=self.rsi_length)

    def next(self):
        super().next()
        slatr = self.slcoef*self.data.ATR[-1]
        TPSLRatio = self.TPSLRatio

        # if len(self.trades)>0:
        #     if self.trades[-1].is_long and self.data.RSI[-1]>=90:
        #         self.trades[-1].close()
        #     elif self.trades[-1].is_short and self.data.RSI[-1]<=10:
        #         self.trades[-1].close()

        if self._signal==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self._signal==1 and len(self.trades)==0:         
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(df, MyStrategy, cash=250, margin=1/30)

/tmp/ipykernel_2812/4216055689.py:33: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



In [40]:
bt.run()

Start                                     0.0
End                                    9998.0
Duration                               9998.0
Exposure Time [%]                   31.923192
Equity Final [$]                   399.681438
Equity Peak [$]                    404.314027
Return [%]                          59.872575
Buy & Hold Return [%]               -4.364162
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -11.228536
Avg. Drawdown [%]                   -1.380149
Max. Drawdown Duration                 2324.0
Avg. Drawdown Duration             104.641975
# Trades                                560.0
Win Rate [%]                        44.642857
Best Trade [%]                       0.377303
Worst Trade [%]                     -0.252185
Avg. Trade [%]                    

In [41]:
bt.plot()

GridPlot(id='p1707', ...)